In [1]:
!pip install azureml-defaults scikit-learn
!pip install azureml-widgets
!pip install azureml-train
!pip install azureml-train-automl
!pip install "azureml-core~=1.60.0"
!pip install "azureml-telemetry~=1.60.0"
!pip uninstall ipywidgets -y
!pip install "ipywidgets<8.0.0,>=7.0.0"
!pip uninstall psutil -y
!pip install "psutil<5.9.4,>=5.2.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 7.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: psutil
    Found existing installation: psutil 5.2.2
    Uninstalling psutil-5.2.2:
      Successfully uninstalled psutil-5.2.2
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-nvdashboard 0.13.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
jupyter-resource-usage 0.7.2 requires psutil~=5.6, but you have psutil 7.1.3 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatibl

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-291922
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-291922


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "myCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    my_compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except:
    my_compute_cluster_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    my_compute_cluster = ComputeTarget.create(ws, cluster_name, my_compute_cluster_config)

my_compute_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.1,1),
        '--max_iter': choice(10,20,50)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=cluster_name,
    environment=sklearn_env    
    )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [39]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_826b0e35-c615-4f57-9eab-852fd0e93385
Web View: https://ml.azure.com/runs/HD_826b0e35-c615-4f57-9eab-852fd0e93385?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-291922/workspaces/quick-starts-ws-291922&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-12-06T00:39:44.0545312Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-12-06T00:39:44.3082715Z][SCHEDULER][INFO]Scheduling job, id='HD_826b0e35-c615-4f57-9eab-852fd0e93385_0' 
[2025-12-06T00:39:44.4421682Z][SCHEDULER][INFO]Scheduling job, id='HD_826b0e35-c615-4f57-9eab-852fd0e93385_1' 
[2025-12-06T00:39:44.4880961Z][SCHEDULER][INFO]Scheduling job, id='HD_826b0e35-c615-4f57-9eab-852fd0e93385_2' 
[2025-12-06T00:39:44.6063674Z][SCHEDULER][INFO]Scheduling job, id='HD_826b0e35-c615-4f57-9eab-852fd0e93385_3' 
[2025-12-06T00:39:45.0156837Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_826b0e35-c615-4f57-9eab-852fd0e93385_2' 
[2025-12-06T00:3

{'runId': 'HD_826b0e35-c615-4f57-9eab-852fd0e93385',
 'target': 'myCluster',
 'status': 'Completed',
 'startTimeUtc': '2025-12-06T00:39:42.595081Z',
 'endTimeUtc': '2025-12-06T00:45:16.193775Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0535663e-5e8d-476a-aebb-7c6dd72c31db',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1030-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_826b0e35-c615-4f57-9eab-852fd0e93385_3',
  'score': '0.8878686327077748',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_826b0e35-c615-4f57-9eab-852fd0e93385_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-trai

In [29]:
!pip install joblib

In [40]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 
best_run_file_names = best_run.get_file_names()

print('Best Run Id: ', best_run.id) 
print('Best Run Metrics: ', best_run_metrics) 
print('Best Run Accuracy:', best_run_metrics['Accuracy']) 
print('Best Run File Names:', best_run_file_names) 

Best Run Id:  HD_826b0e35-c615-4f57-9eab-852fd0e93385_3
Best Run Metrics:  {'Regularization Strength:': 0.1, 'Max iterations:': 20, 'Accuracy': 0.8878686327077748}
Best Run Accuracy: 0.8878686327077748
Best Run File Names: ['logs/azureml/dataprep/0/rslex.log.2025-12-06-00', 'outputs/.amlignore', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [41]:
hd_model = best_run.register_model(model_name='hd_model', model_path='outputs/model.joblib')

In [20]:
from azureml.core import Workspace, Dataset

subscription_id = 'f5091c60-1c3c-430f-8d81-d802f6bf2414'
resource_group = 'aml-quickstarts-291922'
workspace_name = 'quick-starts-ws-291922'

workspace = Workspace(subscription_id, resource_group, workspace_name)

ds = Dataset.get_by_name(workspace, name='bankmarketing_train')

In [21]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [23]:
# Submit your automl run
from azureml.core.experiment import Experiment

experiment_automl = Experiment(workspace, "automl")

automl_run = experiment_automl.submit(automl_config)
RunDetails(automl_run).show()

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
2025-12-05:23:21:44,986 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_84e4c008-c571-4659-aa0f-1d8bdaec838a,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025/12/05 23:22:37 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025-12-05:23:56:21,137 INFO     [explanation_client.py:334] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [46]:
# Retrieve and save your best automl model.
automl_run_metrics = automl_run.get_metrics() 
automl_run_file_names = automl_run.get_file_names()

print('Best Run Metrics: ', automl_run_metrics) 
print('Best Run Accuracy:', automl_run_metrics['accuracy']) 
print('Best Run File Names:', automl_run_file_names)

Best Run Metrics:  {'average_precision_score_micro': 0.975769863142806, 'norm_macro_recall': 0.4307956407487046, 'AUC_weighted': 0.9305239053142573, 'accuracy': 0.9079870314518954, 'f1_score_micro': 0.9079870314518954, 'matthews_correlation': 0.5002000593060313, 'precision_score_macro': 0.7905474326109545, 'recall_score_micro': 0.9079870314518954, 'AUC_micro': 0.9749855352706279, 'precision_score_weighted': 0.8985762018397584, 'precision_score_micro': 0.9079870314518954, 'log_loss': 0.260075192507808, 'f1_score_macro': 0.7449028026975134, 'recall_score_macro': 0.7153978203743523, 'AUC_macro': 0.9305239053142573, 'f1_score_weighted': 0.9011344279406173, 'recall_score_weighted': 0.9079870314518954, 'average_precision_score_macro': 0.8035813706592753, 'weighted_accuracy': 0.9581551555446246, 'balanced_accuracy': 0.7153978203743523, 'average_precision_score_weighted': 0.9463454859281444}
Best Run Accuracy: 0.9079870314518954
Best Run File Names: ['outputs/verifier_results.json']


In [48]:
best_automl_run, best_automl_model= automl_run.get_output()
best_automl_model = best_automl_run.register_model(model_name="best_automl_model.pkl", model_path="outputs/")

In [ ]:
my_compute_cluster.delete()
